In [1]:
import pandas as pd

# Load the dataset
file_path = "D:\Projects\Earthquake Prediction\Earthquake_Data.csv"
df = pd.read_csv(file_path)

# Display basic information and the first few rows
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18030 entries, 0 to 18029
Data columns (total 1 columns):
 #   Column                                                                                                                                              Non-Null Count  Dtype 
---  ------                                                                                                                                              --------------  ----- 
 0   Date(YYYY/MM/DD)    Time           Latitude        Longitude           Depth       Mag      Magt     Nst    Gap     Clo     RMS    SRC     EventID  18030 non-null  object
dtypes: object(1)
memory usage: 141.0+ KB


(None,
   Date(YYYY/MM/DD)    Time           Latitude        Longitude           Depth       Mag      Magt     Nst    Gap     Clo     RMS    SRC     EventID
 0  1966/07/01 09:41:21.82  35.9463 -120.4700  12....                                                                                                
 1  1966/07/02 12:08:34.25  35.7867 -120.3265   8....                                                                                                
 2  1966/07/02 12:16:14.95  35.7928 -120.3353   9....                                                                                                
 3  1966/07/02 12:25:06.12  35.7970 -120.3282   9....                                                                                                
 4  1966/07/05 18:54:54.36  35.9223 -120.4585   7....                                                                                                )

In [2]:
# Attempt to split the single column into multiple columns based on spaces
df_cleaned = df.iloc[:, 0].str.split(r'\s+', expand=True)

# Display first few rows to check the structure
df_cleaned.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1966/07/01,09:41:21.82,35.9463,-120.4700,12.26,3.20,Mx,7,171,20,0.02,NCSN,-4540462
1,1966/07/02,12:08:34.25,35.7867,-120.3265,8.99,3.70,Mx,8,86,3,0.04,NCSN,-4540520
2,1966/07/02,12:16:14.95,35.7928,-120.3353,9.88,3.40,Mx,8,89,2,0.03,NCSN,-4540521
3,1966/07/02,12:25:06.12,35.7970,-120.3282,9.09,3.10,Mx,8,101,3,0.08,NCSN,-4540522
4,1966/07/05,18:54:54.36,35.9223,-120.4585,7.86,3.10,Mx,9,161,14,0.04,NCSN,-4540594


In [3]:
# Assign proper column names
df_cleaned.columns = [
    "Date", "Time", "Latitude", "Longitude", "Depth", "Mag", "Magt",
    "Nst", "Gap", "Clo", "RMS", "SRC", "EventID"
]

# Select relevant features and target variable
df_model = df_cleaned[["Latitude", "Longitude", "Nst", "Mag"]]

# Convert data types to numeric
df_model = df_model.astype({"Latitude": float, "Longitude": float, "Nst": int, "Mag": float})

# Display dataset info and first few rows
df_model.info(), df_model.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18030 entries, 0 to 18029
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Latitude   18030 non-null  float64
 1   Longitude  18030 non-null  float64
 2   Nst        18030 non-null  int32  
 3   Mag        18030 non-null  float64
dtypes: float64(3), int32(1)
memory usage: 493.1 KB


(None,
    Latitude  Longitude  Nst  Mag
 0   35.9463  -120.4700    7  3.2
 1   35.7867  -120.3265    8  3.7
 2   35.7928  -120.3353    8  3.4
 3   35.7970  -120.3282    8  3.1
 4   35.9223  -120.4585    9  3.1)

In [11]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Define features and target variable
X = df_model[["Latitude", "Longitude", "Nst"]]
y = df_model["Mag"]

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

mae, rmse


(0.3156525998451205, 0.42087408330380904)

In [18]:
# Function to make predictions on user input
def predict_earthquake(latitude, longitude, nst):
    input_data = scaler.transform([[latitude, longitude, nst]])
    predicted_mag = model.predict(input_data)[0]
    return round(predicted_mag, 2)

# Function to take user input and predict earthquake magnitude
def user_input_prediction():
    try:
        latitude = float(input("Enter Latitude: "))
        longitude = float(input("Enter Longitude: "))
        nst = int(input("Enter Number of Stations (Nst): "))
        
        predicted_magnitude = predict_earthquake(latitude, longitude, nst)
        print(f"Predicted Earthquake Magnitude: {predicted_magnitude}")
    
    except ValueError:
        print("Invalid input! Please enter numerical values for Latitude, Longitude, and Nst.")

# Call the function to allow user input
user_input_prediction()


Predicted Earthquake Magnitude: 3.39


c:\Users\aadik\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
# import pickle

# # Save the trained model
# with open("earthquake_model1.pkl", "wb") as model_file:
#     pickle.dump(model, model_file)

In [19]:
import joblib

joblib.dump(model, 'model.pkl')

['model.pkl']